In [61]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import os
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed, Lambda
from keras.models import Model
from keras import initializers
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model
import pandas as pd
import pickle as cPickle
from collections import defaultdict
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
MAX_SENTENCE_LENGTH = 100
MAX_SENTS = 15
NB_WORDS = 20000
EMBEDDING_DIMENSION = 100
VAL_SPLIT = 0.2

In [63]:
def string_cleaner(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [64]:
data_train = pd.read_csv('labeledTrainData.tsv', sep='\t')

In [65]:
print (data_train.shape)

(25000, 3)


In [66]:
review = []
w_label = []
texts_tt = []
for idx in range(data_train.review.shape[0]):
    text_tt = BeautifulSoup(data_train.review[idx])
    text_tt = string_cleaner(text_tt.get_text().encode('ascii', 'ignore').decode('ascii'))
    texts_tt.append(text_tt)
    sents = tokenize.sent_tokenize(text_tt)
    review.append(sents)
    w_label.append(data_train.sentiment[idx])

In [67]:
tokenizer = Tokenizer(nb_words=NB_WORDS)
tokenizer.fit_on_texts(texts_tt)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [68]:
w_data = np.zeros((len(texts_tt), MAX_SENTENCE_LENGTH), dtype='int32')
for i, con in enumerate(texts_tt):
    wordTokens = text_to_word_sequence(con)
    j = 0
    for _, W in enumerate(wordTokens):
        if j < MAX_SENTENCE_LENGTH and tokenizer.word_index[W] < NB_WORDS:
            w_data[i, j] = tokenizer.word_index[W]
            j = j + 1

In [69]:
w_index = tokenizer.word_index

In [70]:
print('There are a total of %s unique tokens.' % len(w_index))

There are a total of 80566 unique tokens.


In [71]:
w_label = to_categorical(np.asarray(w_label))

In [72]:
print('The shape of the data tensor is:', w_data.shape)
print('The shape of the label tensor is:', w_label.shape)

The shape of the data tensor is: (25000, 100)
The shape of the label tensor is: (25000, 2)


In [73]:
inx = np.arange(w_data.shape[0])
np.random.shuffle(inx)
w_data = w_data[inx]
w_label = w_label[inx]
validation_samples = int(VAL_SPLIT * w_data.shape[0])

In [74]:
x_train = w_data[:-validation_samples]
y_train = w_label[:-validation_samples]
x_val = w_data[-validation_samples:]
y_val = w_label[-validation_samples:]

In [75]:
print('Positive & Negative reviews in train & val set respectively are: ')
print (y_train.sum(axis=0))
print (y_val.sum(axis=0))

Positive & Negative reviews in train & val set respectively are: 
[10025.  9975.]
[2475. 2525.]


In [76]:
from google.colab import drive
drive.mount('/content/gdrive')
GLOVE_DIR = "."
embeddings_INDEX = {}
f = open(os.path.join(GLOVE_DIR, '/content/gdrive/My Drive/glove/glove.6B.100d.txt'))
for line in f:
    values = line.split()
    W = values[0]
    coeficients = np.asarray(values[1:], dtype='float32')
    embeddings_INDEX[W] = coeficients
f.close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [77]:
print('There are a total number of %s word vectors.' % len(embeddings_INDEX))

There are a total number of 400000 word vectors.


In [78]:
embedding_MAT = np.random.random((len(w_index) + 1, EMBEDDING_DIMENSION))
for W, i in w_index.items():
    embedding_VEC = embeddings_INDEX.get(W)
    if embedding_VEC is not None:
        embedding_MAT[i] = embedding_VEC

In [79]:
embedding_layer = Embedding(len(w_index) + 1,
                            EMBEDDING_DIMENSION,
                            weights=[embedding_MAT],
                            input_length=MAX_SENTENCE_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [80]:
class Attention_Layer(Layer):
    def __init__(self, attention_dim, **kwargs):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(Attention_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self._W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self._b = K.variable(self.init((self.attention_dim,)))
        self._u = K.variable(self.init((self.attention_dim, 1)))
        self._trainable_weights = [self._W, self._b, self._u]
        super(Attention_Layer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        _uit = K.tile(K.expand_dims(self._W, axis=0), (K.shape(x)[0], 1, 1))
        _uit = tf.matmul(x, _uit)
        _uit = K.tanh(K.bias_add(_uit, self._b))
        _ait = K.dot(_uit, self._u)
        _ait = K.squeeze(_ait, -1)
        _ait = K.exp(_ait)

        if mask is not None:
            _ait *= K.cast(mask, K.floatx())
        
        _ait /= K.cast(K.sum(_ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        _ait = K.expand_dims(_ait)
        weighted_input = x * _ait
        output = K.sum(weighted_input, axis=1)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[2])

    def get_config(self):
        config = {'attention_dim': self.attention_dim}
        base_config = super(Attention_Layer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [81]:
sent_input = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_seq = embedding_layer(sent_input)
L_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_seq)
L_dense = TimeDistributed(Dense(200))(L_lstm)
L_att = Attention_Layer(100)(L_dense)

In [82]:
pred = Dense(2, activation='softmax')(L_att)

In [83]:
model = Model(sent_input, pred)

In [84]:
print(x_train.shape)

(20000, 100)


In [85]:
print(y_train.shape)

(20000, 2)


In [86]:
print(x_val.shape)

(5000, 100)


In [87]:
print(y_val.shape)

(5000, 2)


In [88]:
plot_model(model, to_file="Model.png")

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("********MODEL FITTING - HAN**************")
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=50,verbose=2)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 100)          8056700   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100, 200)          121200    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 100, 200)          40200     
_________________________________________________________________
attention__layer_5 (Attentio (None, 200)               20200     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 402       
Total params: 8,238,702
Trainable params: 8,238,702
Non-trainable params: 0
_________________________________________________